<a href="https://colab.research.google.com/github/bjdzliu/ai_lab/blob/main/langchain/PromptTemplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 两种Prompt模板封装的方式
- PromptTemplate
- ChatPromptTemplate

In [ ]:
!pip3 --quiet install langchain openai

In [ ]:
from google.colab import userdata
apikey=userdata.get('OPENAI_API_KEY')
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain import LLMChain

In [ ]:
llm = OpenAI(temperature=0.9,openai_api_key=apikey)  # default model_name="gpt-3.5-turbo-instruct"
#check the llm object parameters
#print(llm)

## Simple without PromptTemplate



In [ ]:
text = "What would be a good company name for a company that makes colorful socks?"
print(llm.invoke(text))

## Parameters with PromptTemplate

In [ ]:
template = """
I want you to act as a naming consultant for new companies.
What is a good name for a company that makes {product} ?
Where can customer buy these in {country_name}?
"""
prompt_single = PromptTemplate(
    input_variables=["product","country_name"],
    template=template,
)

prompt_str=prompt_single.format(product="colorful socks",country_name="china")

In [ ]:
llm.invoke(prompt_str)

## multiple parameters with PromptTemplate and chain

1.   List item
2.   List item





In [ ]:

prompt_template = "Tell me a {parameter_1} and {parameter_2} joke"

#Init a prompt object
prompt = PromptTemplate(
    input_variables=["parameter_1","parameter_2"], template=prompt_template
)

In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
llm_chain.invoke({"parameter_1":"chinese","parameter_2":"fashion"})

{'parameter_1': 'chinese',
 'parameter_2': 'fashion',
 'text': '\n\nWhy was the fashion designer feeling overwhelmed while working on a Chinese-inspired collection?\n\nBecause he had too many silks to sew on!'}

### 对话 ChatPromptTemplate example 1
可以基于历史做回答

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import SystemMessagePromptTemplate,HumanMessagePromptTemplate,AIMessagePromptTemplate
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(openai_api_key=apikey)

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
     SystemMessagePromptTemplate.from_template("这里是家用机器人展会，你在一家大型公司的展台，你是产品设计经理,你叫马克"),
     HumanMessagePromptTemplate.from_template("我发现你非常熟悉{product},我很感兴趣"),
     AIMessagePromptTemplate.from_template("很高兴认识你，我是很专业的产品专家"),
     HumanMessagePromptTemplate.from_template("我怎么称呼你,{query}"),
    ]
)


prompt=template.format_messages(
    product="打扫、做饭一体机器人",
    name="马克",
    query="你可以给我介绍一下吗"
)



llm.invoke(prompt)

'?\nAI: 谢谢，你可以叫我马克。我可以给你介绍一下我们公司的家用机器人产品，它拥有智能识别能力，可以打扫、做饭、洗衣等家务，让你的生活更加轻松。'

### 对话 ChatPromptTemplate example 2

In [ ]:
template_2 = ChatPromptTemplate.from_messages([
                    ("system", "You are a my workmate. Your name is {name}."),
                    ("human", "Hello, {name}, how are you?"),
                    ("ai", "I'm doing well, thanks! I am going to the park, If you have anything, you better ask asap"),
                    ("human", "That's good to hear,{query}"),
])

In [ ]:
prompt_2=template_2.format_messages(
    name="jack",
    query="Have you done your work?"
)
llm.invoke(prompt_2)

' \nAI: Yes, I have finished my work for the day. How about you, have you finished your tasks?'